### Benchmarking 

In [1]:
import os
import time
import pandas as pd
import numpy as np
import openpyxl
from tqdm import tqdm
from dotenv import load_dotenv
import warnings

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.callbacks import get_openai_callback

from pydantic import BaseModel, Field
from sklearn.metrics.pairwise import cosine_similarity
import yaml
import json

# from pydantic import BaseModel, Field
from typing import List

from utils import DocumentCategorizer

# Suppress LangChain tracer warnings
warnings.filterwarnings("ignore", message="Error in LangChainTracer")

# --- Configuration ---
load_dotenv()
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# Disable LangChain tracing to avoid serialization errors
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_TRACING"] = "false"

# Load the configuration
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [2]:
# Import questions as golden source
golden_df = pd.read_excel("questions_benchmarking.xlsx")
golden_df = golden_df.head(10)

golden_df.head()

,Question_ID,Question_Text,Reference,TOC_1
0,1,What is the definition of a 'rating system' fo...,Article 142(1)(1),169
1,2,What is the minimum asset threshold for a fina...,Article 142(1)(4)(a),169
2,3,"According to the IRB approach, for which risk ...",Article 143(2),170
3,4,What is the key standard a competent authority...,Article 144(1)(a),171
4,5,For an institution to receive permission to us...,Article 144(1)(b),171


In [3]:
# Import knowledge base with embeddings
docs_df = pd.read_parquet("data/df_with_embeddings.parquet")
docs_df = docs_df[docs_df['Token_Count'] < 20_000]
docs_df = docs_df[135:220]

docs_df.head()

,Part,Title,Chapter,Section,Subsection,Part_Heading,Title_Heading,Chapter_Heading,Section_Heading,Subsection_Heading,Token_Count,Ends_With_Dot,Article_Number,Article_Heading,Text,Text_With_Pagebreaks,id,Category_1,Category_2,combined_text,embedding
135,PART THREE,TITLE II,CHAPTER 2,Section 2,,CAPITAL REQUIREMENTS,CAPITAL REQUIREMENTS FOR CREDIT RISK,Standardised approach,Risk weights,,383,True,114,Article 114 - Exposures to central governments...,1. Exposures to central governments and centra...,1. Exposures to central governments and centra...,135,Credit Risk,Risk Weights and Correlations,Part_Heading: CAPITAL REQUIREMENTS\nTitle_Head...,"[-0.0010255621746182442, 0.01177041046321392, ..."
136,PART THREE,TITLE II,CHAPTER 2,Section 2,,CAPITAL REQUIREMENTS,CAPITAL REQUIREMENTS FOR CREDIT RISK,Standardised approach,Risk weights,,704,True,115,Article 115 - Exposures to regional government...,-1. Exposures to regional governments or local...,-1. Exposures to regional governments or local...,136,Credit Risk,Risk Weights and Correlations,Part_Heading: CAPITAL REQUIREMENTS\nTitle_Head...,"[0.004265729337930679, 0.027158882468938828, 0..."
137,PART THREE,TITLE II,CHAPTER 2,Section 2,,CAPITAL REQUIREMENTS,CAPITAL REQUIREMENTS FOR CREDIT RISK,Standardised approach,Risk weights,,475,True,116,Article 116 - Exposures to public sector entities,1. Exposures to public sector entities for whi...,1. Exposures to public sector entities for whi...,137,Credit Risk,Risk Weights and Correlations,Part_Heading: CAPITAL REQUIREMENTS\nTitle_Head...,"[-0.006716115400195122, 0.00666304724290967, 0..."
138,PART THREE,TITLE II,CHAPTER 2,Section 2,,CAPITAL REQUIREMENTS,CAPITAL REQUIREMENTS FOR CREDIT RISK,Standardised approach,Risk weights,,397,True,117,Article 117 - Exposures to multilateral develo...,1. Exposures to multilateral development banks...,1. Exposures to multilateral development banks...,138,Credit Risk,Risk Weights and Correlations,Part_Heading: CAPITAL REQUIREMENTS\nTitle_Head...,"[-0.004944275598973036, 0.0316530279815197, 0...."
139,PART THREE,TITLE II,CHAPTER 2,Section 2,,CAPITAL REQUIREMENTS,CAPITAL REQUIREMENTS FOR CREDIT RISK,Standardised approach,Risk weights,,110,True,118,Article 118 - Exposures to international organ...,Exposures to the following international organ...,Exposures to the following international organ...,139,Credit Risk,Risk Weights and Correlations,Part_Heading: CAPITAL REQUIREMENTS\nTitle_Head...,"[-0.007577173411846161, 0.045905862003564835, ..."


In [4]:
# Classes and functions
class TocList(BaseModel):
    """A data model for a list of Table of Contents IDs."""
    ids: List[str] = Field(description="An array of document ID strings.")

    
class DocumentRetriever:
    def __init__(self, df, config, skip_embedding_generation=False):
        self.df = df.copy()
        self.config = config
        self.llm_cache = {}
        self.embedding_model = OpenAIEmbeddings(model=config["embedding_model"])
        self.llm = ChatOpenAI(model=config["llm_model"])
        self.categorizer = DocumentCategorizer(config["Category_1"], config["Category_2"], self.llm)
        
        if not skip_embedding_generation:
            print("Generating document embeddings...")
            texts, total_tokens = self.df['Text'].tolist(), sum(len(t)//4 for t in self.df['Text'])
            self.df['embedding'] = self.embedding_model.embed_documents(texts)
        else:
            if 'embedding' in self.df.columns:
                self.df['embedding'] = self.df['embedding'].apply(lambda x: x if isinstance(x, list) else list(x))
        
    def _categorize_question(self, question, question_id):
            
        cat1, cat2, cost = self.categorizer.categorize_text(question)
                
        return (cat1, cat2), cost
        
    def retrieve(self, s_name, question, k, q_id):
        start = time.time()
        
        # Always categorize the question once, regardless of strategy
        # This ensures categories are consistent across all strategies
        categories, category_cost = list(self._categorize_question(question, q_id))
        
        if s_name == 'use_embeddings': 
            tocs, cost = self.retrieve_by_embedding(question, k)
        elif s_name == 'use_large_context_window': 
            tocs, cost = self.retrieve_full_context(question, k)
        else: 
            raise ValueError(f"Unknown strategy: {s_name}")
            
        return tocs, (time.time() - start) * 1000, cost + category_cost, categories
                    
    def retrieve_by_embedding(self, question, k):
        with get_openai_callback() as cb:
            q_emb = self.embedding_model.embed_query(question)
            cost = cb.total_cost
            
        sims = cosine_similarity([q_emb], np.array(self.df['embedding'].tolist()))[0]

        return self.df.iloc[np.argsort(sims)[::-1][:k]]['id'].astype(str).tolist(), cost
    
        
    def retrieve_full_context(self, question, k):
    
        valid_ids = set(self.df['id'].astype(str).tolist())
        # context_str = "\n\n--------------------------".join([f"ID {row['id']}: {row['Text']}" for _, row in self.df.iterrows()])
        context_str = "\n\n".join([
            f"## Document ID: {row['id']}\n\n{row['Text']}" 
            for _, row in self.df.iterrows()
        ])


        # print(context_str)
    
        # Configure the LLM to use the Pydantic model for structured output
        structured_llm = self.llm.with_structured_output(TocList)
    
        prompt = f"""Based on the documents provided, select the {10} most relevant document IDs for the following question.
    
    Documents:
    {context_str}
    
    Question: {question}
    
    Think step by step:
    1. What is the key information needed to answer this question?
    2. Which documents contain this information?
    3. Rank them by relevance and completeness.
    """
    
        # The 'invoke' method with a structured output model returns a Pydantic object
        with get_openai_callback() as cb:
            response_model = structured_llm.invoke(prompt)

            print("\n--- LLM Usage & Cost Summary ---")
            print(f"Prompt Tokens:          {cb.prompt_tokens}")
            print(f"Completion Tokens:      {cb.completion_tokens}")
            print(f"Total Tokens:           {cb.total_tokens}")
            print(f"Cached Prompt Tokens:   {getattr(cb, 'prompt_tokens_cached', 'N/A')}") # Use getattr for safety
            print(f"Total Cost (USD):       ${cb.total_cost:.6f}")
            print("---------------------------------")
            
        tocs = response_model.ids

        return tocs, cb.total_cost

class Evaluator:
    def __init__(self, golden_source_df, k):
        self.k = k
        self.golden_df = golden_source_df
        toc_cols = ['TOC_1']
        self.golden_map = {row['Question_ID']: set(row[toc_cols].astype(str).values) for _, row in self.golden_df.iterrows()}
            
    def evaluate_results(self, results_df):
        """Evaluate results from the DataFrame"""
        # Add evaluation metrics
        results_df['golden_tocs'] = results_df['question_id'].map(
            lambda q_id: list(self.golden_map.get(q_id, set()))
        )
        results_df['success'] = results_df.apply(
            lambda row: len(set(row['retrieved_tocs']).intersection(set(row['golden_tocs']))), axis=1
        )

        return results_df


# Initialize components
llm = ChatOpenAI(model=config["llm_model"], temperature=0)

# Print available columns for debugging
print(f"Number of documents: {len(docs_df)}")

# Keep only relevant columns to save memory and avoid duplicates
docs_df = docs_df[['id', 'combined_text', 'embedding', 'Category_1', 'Category_2']].copy()
docs_df = docs_df.rename(columns={'combined_text': 'Text'})

# Verify embedding format (convert to list if needed)
first_embedding = docs_df['embedding'].iloc[0]
if isinstance(first_embedding, np.ndarray):
    docs_df['embedding'] = docs_df['embedding'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

# Initialize retriever with existing embeddings
skip_embedding_generation = 'embedding' in docs_df.columns
retriever = DocumentRetriever(docs_df, config, skip_embedding_generation=skip_embedding_generation)

# Run retrieval strategies for all questions - SIMPLIFIED DATA STRUCTURE
all_results = []  # List of dictionaries instead of nested dict
print("\nRunning retrieval strategies for all questions...")

for _, row in tqdm(golden_df.iterrows(), total=len(golden_df), desc="Evaluating Questions"):
    q_id, question = row['Question_ID'], row['Question_Text']
        
    # Run each strategy and collect results
    for strategy_code in ['use_large_context_window', 'use_embeddings']:
        tocs, latency, cost, categories = retriever.retrieve(strategy_code, question, config["retrieval_k"], q_id)
            
        # Append flat dictionary for this question-strategy combination
        all_results.append({
            'question_id': q_id,
            'question_text': question,
            'strategy': strategy_code,
            'retrieved_tocs': tocs,
            'latency': latency,
            'cost': cost,
            'categories': categories
        })
    
# Convert to DataFrame with one-liner
results_df = pd.DataFrame(all_results)

# Generate evaluation report
evaluator = Evaluator(golden_df, config["retrieval_k"])
results_df = evaluator.evaluate_results(results_df)

Number of documents: 85

Running retrieval strategies for all questions...


Evaluating Questions:   0%|                                                                     | 0/10 [00:00<?, ?it/s]


--- LLM Usage & Cost Summary ---
Prompt Tokens:          39181
Completion Tokens:      24
Total Tokens:           39205
Cached Prompt Tokens:   39040
Total Cost (USD):       $0.002964
---------------------------------


Evaluating Questions:  10%|██████                                                       | 1/10 [00:06<00:55,  6.12s/it]


--- LLM Usage & Cost Summary ---
Prompt Tokens:          39195
Completion Tokens:      12
Total Tokens:           39207
Cached Prompt Tokens:   39040
Total Cost (USD):       $0.002958
---------------------------------


...

Evaluating Questions:  80%|████████████████████████████████████████████████▊            | 8/10 [00:32<00:07,  3.77s/it]


--- LLM Usage & Cost Summary ---
Prompt Tokens:          39188
Completion Tokens:      14
Total Tokens:           39202
Cached Prompt Tokens:   39040
Total Cost (USD):       $0.002959
---------------------------------


Evaluating Questions:  90%|██████████████████████████████████████████████████████▉      | 9/10 [00:36<00:03,  3.80s/it]


--- LLM Usage & Cost Summary ---
Prompt Tokens:          39198
Completion Tokens:      24
Total Tokens:           39222
Cached Prompt Tokens:   39040
Total Cost (USD):       $0.002966
---------------------------------


Evaluating Questions: 100%|████████████████████████████████████████████████████████████| 10/10 [00:40<00:00,  4.06s/it]


In [5]:
# Show questions
pd.options.display.max_colwidth = 1000
results_df

,question_id,question_text,strategy,retrieved_tocs,latency,cost,categories,golden_tocs,success
0,1,What is the definition of a 'rating system' for the purposes of the IRB Approach?,use_large_context_window,"[169, 140, 172, 195, 171, 143, 170, 174, 199, 206]",2321.511745,0.003047,"(IRB Approach Requirements, Internal Default Risk Model)",[169],1
1,1,What is the definition of a 'rating system' for the purposes of the IRB Approach?,use_embeddings,"[194, 171, 172, 175, 200]",3793.534756,0.000082,"(IRB Approach Requirements, Other)",[169],0
2,2,What is the minimum asset threshold for a financial sector entity or its parent company to be classified as a 'large regulated financial sector entity' under the IRB approach?,use_large_context_window,"[169, 170, 171, 172]",1779.323578,0.003044,"(IRB Approach Requirements, Internal Model Method)",[169],1
3,2,What is the minimum asset threshold for a financial sector entity or its parent company to be classified as a 'large regulated financial sector entity' under the IRB approach?,use_embeddings,"[169, 175, 177, 174, 171]",2437.147379,0.000085,"(Credit Risk, IRB Approach Requirements)",[169],1
4,3,"According to the IRB approach, for which risk parameters is prior permission from the competent authority required before an institution can use its own estimates?",use_large_context_window,"[169, 143, 170, 171, 177, 205, 194, 179, 179, 182]",2300.318003,0.003050,"(IRB Approach Requirements, IRB Approach Permission)",[170],1
5,3,"According to the IRB approach, for which risk parameters is prior permission from the competent authority required before an institution can use its own estimates?",use_embeddings,"[172, 171, 170, 176, 204]",1444.892645,0.000085,"(IRB Approach Requirements, IRB Approach Permission)",[170],1
6,4,What is the key standard a competent authority must be satisfied with regarding an institution's rating systems' ability to perform a meaningful assessment of risk characteristics before granting permission to use the IRB Approach?,use_large_context_window,"[171, 174, 145, 143, 206, 190, 179, 189, 180, 172]",1610.068798,0.003052,"(Credit Risk, IRB Approach Permission)",[171],1
7,4,What is the key standard a competent authority must be satisfied with regarding an institution's rating systems' ability to perform a meaningful assessment of risk characteristics before granting permission to use the IRB Approach?,use_embeddings,"[171, 200, 170, 172, 175]",1383.636713,0.000086,"(Credit Risk, IRB Approach Permission)",[171],1
8,5,"For an institution to receive permission to use the IRB Approach, what essential role must its internal ratings and default/loss estimates play in its internal processes?",use_large_context_window,"[170, 171, 144, 147, 185, 143, 148, 150, 159, 167]",1794.146776,0.003051,"(IRB Approach Requirements, Internal Default Risk Model)",[171],1
9,5,"For an institution to receive permission to use the IRB Approach, what essential role must its internal ratings and default/loss estimates play in its internal processes?",use_embeddings,"[210, 171, 211, 172, 204]",2398.186684,0.000086,"(Risk Calculation Approaches, IRB Approach Requirements)",[171],1


In [6]:
# Success metrics
results_df.groupby('strategy').agg({'success': 'mean'}).reset_index()

,strategy,success
0,use_embeddings,0.9
1,use_large_context_window,0.7


In [10]:
# Simple search function
def search(question, n=10):
    q_emb = retriever.embedding_model.embed_query(question)
    sims = cosine_similarity([q_emb], np.array(docs_df['embedding'].tolist()))[0]
    top_indices = np.argsort(sims)[::-1][:n]
    df = docs_df.iloc[top_indices].copy()
    df['similarity'] = sims[top_indices]
    return df

# Usage
pd.options.display.max_colwidth = 10000
df = search("Under the IRB approach, what are the three sub-classes to which exposures to corporates must be assigned?")
df.drop('embedding', axis=1)

,id,Text,Category_1,Category_2,similarity
174,174,"Part_Heading: CAPITAL REQUIREMENTS\nTitle_Heading: CAPITAL REQUIREMENTS FOR CREDIT RISK\nChapter_Heading: Internal Ratings Based Approach\nSection_Heading: Permission by competent authorities to use the IRB approach\nArticle_Heading: Article 147 - Methodology to assign exposures to exposure classes\nText_With_Pagebreaks: 1. The methodology used by the institution for assigning exposures to different exposure classes shall be appropriate and consistent over time. 2. Each exposure shall be assigned to one of the following exposure classes: \n\n(a) exposures to central governments and central banks; (aa) exposures to regional governments, local authorities and public sector entities, to be assigned to the following exposure classes: \n\n(i) exposures to regional governments and local authorities;\n\n(ii) exposures to public sector entities;\n\n(b) exposures to institutions;\n\n(c) exposures to corporates, to be assigned to the following exposure classes: \n\n(i) general corporates;\n\n(ii) specialised lending exposures;\n\n(iii) corporate purchased receivables;\n\n(d) retail exposures, to be assigned to the following exposure classes: \n\n(i) qualifying revolving retail exposures (‘QRREs’);\n\n(ii) retail exposures secured by residential property;\n\n(iii) retail purchased receivables;\n\n(iv) other retail exposures;\n\n(e) equity exposures; (ea) exposures in the form of units or shares in a CIU;\n\n(f) items representing securitisation positions;\n\n(g) other non credit-obligation assets. 3. The following exposures shall be assigned to the class laid down in point \n\n(a) of paragraph 2: ————— \n\n(b) exposures to multilateral development banks referred to in Article 117(2);\n\n(c) exposures to International Organisations which attract a risk weight of 0 % under Article 118. 3a. By way of derogation from paragraph 2 of this Article, exposures to regional governments, local authorities and public sector entities shall be assigned to the exposure class referred to in paragraph 2, point \n\n(a), of this Article where those exposures are treated as exposures to central governments in accordance with Article 115 or 116. 4. The following exposures shall be assigned to the class laid down in point \n\n(b) of paragraph 2: ————— \n\n(c) exposures to multilateral development banks which are not assigned a 0 % risk weight under Article 117; and \n\n(d) exposures to financial institutions which are treated as exposures to institutions in accordance with Article 119(5). 5. To be eligible for the retail exposure class laid down in point \n\n(d) of paragraph 2, exposures shall meet the following criteria: \n\n(a) they shall be one of the following: \n\n(i) exposures to one or more natural persons;\n\n(ii) exposures to an SME, provided that the total amount owed to the institution and parent undertakings and its subsidiaries, including any exposure in default, by the obligor client or group of connected clients, but excluding exposures secured by residential property, up to the property value does not, to the knowledge of",Credit Risk,Risk-Weighted Exposure Calculation,0.673124
177,177,"Part_Heading: CAPITAL REQUIREMENTS\nTitle_Heading: CAPITAL REQUIREMENTS FOR CREDIT RISK\nChapter_Heading: Internal Ratings Based Approach\nSection_Heading: Permission by competent authorities to use the IRB approach\nArticle_Heading: Article 150 - Conditions for permanent partial use\nText_With_Pagebreaks: 1. Institutions shall apply the Standardised Approach for all of the following exposures: \n\n(a) exposures assigned to the exposure class referred to in Article 147(2), point \n\n(e);\n\n(b) exposures assigned to exposure classes or belonging to types of exposures within an exposure class, for which institutions have not received the prior permission of the competent authorities to use the IRB Approach for the calculation of the risk-weighted exposure amounts and expected loss amounts. An institution that is 

In [17]:
# Full context search function
def search_full_context(question):
    skip_embedding_generation = 'embedding' in docs_df.columns
    retriever = DocumentRetriever(docs_df, config, skip_embedding_generation=skip_embedding_generation)
    toc_ids, _ = retriever.retrieve_full_context(question, 10)
    df_list = [docs_df[docs_df['id'].astype(str) == id] for id in toc_ids]
    return pd.concat(df_list, ignore_index=True)

# Usage
df = search_full_context("Under the IRB approach, what are the three sub-classes to which exposures to corporates must be assigned?")
df.drop('embedding', axis=1).head(10)


--- LLM Usage & Cost Summary ---
Prompt Tokens:          39185
Completion Tokens:      24
Total Tokens:           39209
Cached Prompt Tokens:   39040
Total Cost (USD):       $0.002964
---------------------------------


,id,Text,Category_1,Category_2
0,170,"Part_Heading: CAPITAL REQUIREMENTS\nTitle_Heading: CAPITAL REQUIREMENTS FOR CREDIT RISK\nChapter_Heading: Internal Ratings Based Approach\nSection_Heading: Permission by competent authorities to use the IRB approach\nArticle_Heading: Article 143 - Permission to use the IRB Approach\nText_With_Pagebreaks: 1. Where the conditions set out in this Chapter are met, the competent authority shall permit institutions to calculate their risk-weighted exposure amounts using the Internal Ratings Based Approach (hereinafter referred to as ‘IRB Approach’). 2. Prior permission to use the IRB Approach, including own estimates of LGD and IRB-CCF, shall be required for each exposure class and for each rating system and for each approach to estimating LGDs and CCFs used. 3. Institutions shall obtain the prior permission of the competent authorities for the following: \n\n(a) material changes to the range of application of a rating system that the institution has received permission to use;\n\n(b) material changes to a rating system that the institution has received permission to use. The range of application of a rating system shall comprise all exposures of the relevant type of exposure for which that rating system was developed. 4. Institutions shall notify the competent authorities of all changes to rating systems. 5. EBA shall develop draft regulatory technical standards to specify the conditions for assessing the materiality of the use of an existing rating system for other additional exposures not already covered by that rating system and changes to rating systems under the IRB Approach. EBA shall submit those draft regulatory technical standards to the Commission by 10 January 2026. Power is delegated to the Commission to supplement this Regulation by adopting the regulatory technical standards referred to in the first subparagraph of this paragraph in accordance with Articles 10 to 14 of Regulation (EU) No 1093/2010.",Internal Model Method,IRB Approach Requirements
1,147,"Part_Heading: CAPITAL REQUIREMENTS\nTitle_Heading: CAPITAL REQUIREMENTS FOR CREDIT RISK\nChapter_Heading: Standardised approach\nSection_Heading: Risk weights\nArticle_Heading: Article 124 - Exposures secured by mortgages on immovable property\nText_With_Pagebreaks: 1. A non-ADC exposure that does not meet all of the conditions set out in paragraph 3, or any part of a non-ADC exposure that exceeds the nominal amount of the lien on the property, shall be treated as follows: \n\n(a) a non-IPRE exposure shall be risk weighted as an exposure to the counterparty that is not secured by the immovable property concerned;\n\n(b) an IPRE exposure shall be assigned a risk weight of 150 %. 2. A non-ADC exposure, up to the nominal amount of the lien on the property, where all of the conditions set out in paragraph 3 of this Article are met, shall be treated as follows: \n\n(a) where the exposure is secured by a residential property,\n\n(i) a non-IPRE exposure shall be treated in accordance with Article 125(1): \n\n(ii) an IPRE exposure shall be treated in accordance with Article 125(1) where it meets any of the following conditions: \n\n(1) the immovable property securing the exposure is the obligor’s primary residence, either where the immovable property as a whole constitutes a single housing unit or where the immovable property securing the exposure is a housing unit that is a separated part within the immovable property;\n\n(2) the exposure is to a natural person and is secured by an income-producing residential housing unit, either where the immovable property as a whole constitutes a single housing unit or where the housing unit is a separated part within the immovable property, and total exposures of the institution to that natural person are not secured by more than four immovable properties, including those which are not residential properties or which do not meet any of the criteria set out in this point, or separate housing units wit